In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder
  .master("local")
  .appName("schiphol")
  // .config("spark.some.config.option", "some-value")
  .getOrCreate()


import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@10b7d80c


In [3]:
// // visualize dataframes
// // https://github.com/almond-sh/almond/issues/180

// import org.apache.spark.sql._

// implicit class RichDF(val df: DataFrame) {
//     def view(limit:Int = 20, truncate: Int = 20) = {
//         import xml.Utility.escape
//         val data = df.take(limit)
//         val header = df.schema.fieldNames.toSeq        
//         val rows: Seq[Seq[String]] = data.map { row =>
//           row.toSeq.map { cell =>
//             val str = cell match {
//               case null => "null"
//               case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
//               case array: Array[_] => array.mkString("[", ", ", "]")
//               case seq: Seq[_] => seq.mkString("[", ", ", "]")
//               case _ => cell.toString
//             }
//             if (truncate > 0 && str.length > truncate) {
//               // do not show ellipses for strings shorter than 4 characters.
//               if (truncate < 4) str.substring(0, truncate)
//               else str.substring(0, truncate - 3) + "..."
//             } else {
//               str
//             }
//           }: Seq[String]
//         }

//         kernel.display.html(s""" <table>
//                 <tr>
//                  ${header.map(h => s"<th>${escape(h)}</th>").mkString}
//                 </tr>
//                 ${rows.map { row =>
//                   s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
//                 }.mkString}
//             </table>
//         """)        
//     }
// }


In [4]:
// Use tooling you think is best fitted for the task (e.g. Docker, public cloud, etc.)
// and give us an explanation why you decided for it. 

In [5]:
// Using the flight routes dataset you will stream the given data into our pipeline, process it and display
// results. Keep in mind the resulting implementation needs to be somehow demoable.

import org.apache.spark.sql.catalyst.ScalaReflection
import spark.implicits._

// columns as described in https://openflights.org/data.html
val cols = Seq("airline", "airlineId", "srcAirport", "srcAirportId", "destAirport", "destAirportId", "codeshare", "stops", "equipment")
// case class FlightRoute(
//     airline: String, // 2-letter (IATA) or 3-letter (ICAO) code of the airline.
//     airlineId: Option[String], // Unique OpenFlights identifier for airline.
//     srcAirport: String, // 3-letter (IATA) or 4-letter (ICAO) code of the source airport.
//     srcAirportId: Option[String], // Unique OpenFlights identifier for source airport.
//     destAirport: String, // 3-letter (IATA) or 4-letter (ICAO) code of the destination airport.
//     destAirportId: Option[String], // Unique OpenFlights identifier for destination airport.
//     codeshare: Option[String], // "Y" if this flight is a codeshare (that is, not operated by Airline, but another carrier), empty otherwise.
//     stops: String, // Number of stops on this flight ("0" for direct)
//     equipment: Option[String], // 3-letter codes for plane type(s) generally used on this flight, separated by spaces
// )
case class FlightRoute(
    airline: String, // 2-letter (IATA) or 3-letter (ICAO) code of the airline.
    airlineId: Option[Int], // Unique OpenFlights identifier for airline.
    srcAirport: String, // 3-letter (IATA) or 4-letter (ICAO) code of the source airport.
    srcAirportId: Option[Int], // Unique OpenFlights identifier for source airport.
    destAirport: String, // 3-letter (IATA) or 4-letter (ICAO) code of the destination airport.
    destAirportId: Option[Int], // Unique OpenFlights identifier for destination airport.
    codeshare: Option[String], // "Y" if this flight is a codeshare (that is, not operated by Airline, but another carrier), empty otherwise.
    stops: Int, // Number of stops on this flight ("0" for direct)
    equipment: Option[String], // 3-letter codes for plane type(s) generally used on this flight, separated by spaces
)

// val encoder = org.apache.spark.sql.Encoders.product[FlightRoute]

// val schema = StructType( Seq (
//     StructField( "airline", StringType, false),
//     StructField( "airlineId", StringType, true),
//     StructField( "srcAirport", StringType, false),
//     StructField( "srcAirportId", StringType, true),
//     StructField( "destAirport", StringType, false),
//     StructField( "destAirportId", StringType, true),
//     StructField( "codeshare", StringType, true),
//     StructField( "stops", StringType, false),
//     StructField( "equipment", StringType, true),
// ))

val df1 = spark.read
// val dataset = spark.read
//     .format("csv")
    .option("header","false")
    // .schema(schema)
    .csv("file:///home/jovyan/routes.dat")
    .toDF(cols: _*)
//     .as[FlightRoute]

// write to folder to use later as stream
df1.write.mode("overwrite").option("header",false).csv("file:///home/jovyan/out/routes")

// The special value \N is used for "NULL" to indicate that no value is available
val df2 = cols.foldLeft(df1)((df: DataFrame, column: String) => df.withColumn(column, when(col(column) === "\\N",lit(null)).otherwise(col(column))))
// cast numeric columns to int
val numericCols = Seq("airlineId", "srcAirportId", "destAirportId", "stops")
val df3 = numericCols.foldLeft(df2)((df: DataFrame, column: String) => df.withColumn(column,col(column).cast(IntegerType)))

// val dataset = df.as(encoder)

// val schema = ScalaReflection.schemaFor[FlightRoute].dataType.asInstanceOf[StructType]
// val newSchema = StructType(schema.map {
//   case StructField( c, t, _, m) ⇒ StructField( c, t, nullable = false, m)
// })
// val dataset = spark.createDataFrame(df.rdd, schema).as[FlightRoute]
val ds = df3.as[FlightRoute]

ds.show(10) // 100000


+-------+---------+----------+------------+-----------+-------------+---------+-----+---------+
|airline|airlineId|srcAirport|srcAirportId|destAirport|destAirportId|codeshare|stops|equipment|
+-------+---------+----------+------------+-----------+-------------+---------+-----+---------+
|     2B|      410|       AER|        2965|        KZN|         2990|     null|    0|      CR2|
|     2B|      410|       ASF|        2966|        KZN|         2990|     null|    0|      CR2|
|     2B|      410|       ASF|        2966|        MRV|         2962|     null|    0|      CR2|
|     2B|      410|       CEK|        2968|        KZN|         2990|     null|    0|      CR2|
|     2B|      410|       CEK|        2968|        OVB|         4078|     null|    0|      CR2|
|     2B|      410|       DME|        4029|        KZN|         2990|     null|    0|      CR2|
|     2B|      410|       DME|        4029|        NBC|         6969|     null|    0|      CR2|
|     2B|      410|       DME|        40

import org.apache.spark.sql.catalyst.ScalaReflection
import spark.implicits._
cols: Seq[String] = List(airline, airlineId, srcAirport, srcAirportId, destAirport, destAirportId, codeshare, stops, equipment)
defined class FlightRoute
df1: org.apache.spark.sql.DataFrame = [airline: string, airlineId: string ... 7 more fields]
df2: org.apache.spark.sql.DataFrame = [airline: string, airlineId: string ... 7 more fields]
numericCols: Seq[String] = List(airlineId, srcAirportId, destAirportId, stops)
df3: org.apache.spark.sql.DataFrame = [airline: string, airlineId: int ... 7 more fields]
ds: org.apache.spark.sql.Dataset[FlightRoute] = [airline: string, airlineId: int ... 7 more fields]


In [6]:
// // val routeTable = (ds: Dataset[FlightRoute]): String =>
// def routeTable(ds: Dataset[FlightRoute]) =
//       records.map {
//         case FlightRoute(name, age, gender)
//           s"<tr><td>airline}</td><td>${airlineId}</td><td>${srcAirport}</td><td>${srcAirportId}</td><td>${destAirport}</td><td>${destAirportId}</td><td>${codeshare}</td><td>${stops}</td><td>${equipment</td></tr>"
//       }.mkString(
//         "<table><tr><th>airline</th><th>airlineId</th><th>srcAirport</th><th>srcAirportId</th><th>destAirport</th><th>destAirportId</th><th>codeshare</th><th>stops</th><th>equipment</th></tr>",
//         "",
//         "</table>")

// routeTable(dataset)


In [7]:
// Create a batch Spark job that read in the routes dataset.
// It should create an overview of the top 10 airports used as source airport.
val top10 = ds.groupBy("srcAirport").count().sort(col("count").desc).limit(10)
top10.show()
// Write the output to a filesystem.
ds.write.mode("overwrite").option("header",true)
   .csv("file:///home/jovyan/out/routes.csv")
val ds_schema = ds.schema
top10.write.mode("overwrite").option("header",true)
   .csv("file:///home/jovyan/out/top10.csv")


+----------+-----+
|srcAirport|count|
+----------+-----+
|       ATL|  915|
|       ORD|  558|
|       PEK|  535|
|       LHR|  527|
|       CDG|  524|
|       FRA|  497|
|       LAX|  492|
|       DFW|  469|
|       JFK|  456|
|       AMS|  453|
+----------+-----+



top10: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [srcAirport: string, count: bigint]
ds_schema: org.apache.spark.sql.types.StructType = StructType(StructField(airline,StringType,true), StructField(airlineId,IntegerType,true), StructField(srcAirport,StringType,true), StructField(srcAirportId,IntegerType,true), StructField(destAirport,StringType,true), StructField(destAirportId,IntegerType,true), StructField(codeshare,StringType,true), StructField(stops,IntegerType,true), StructField(equipment,StringType,true))


In [8]:
// Use Spark structured streaming to change your job into a streaming job, and use the dataset file
// as a source.

import org.apache.spark._
import org.apache.spark.streaming._
import org.apache.spark.streaming.dstream._
import org.apache.spark.sql.streaming._

val schema = StructType( Seq (
    StructField("airline", StringType, false),
    StructField("airlineId", StringType, true),
    StructField("srcAirport", StringType, false),
    StructField("srcAirportId", StringType, true),
    StructField("destAirport", StringType, false),
    StructField("destAirportId", StringType, true),
    StructField("codeshare", StringType, true),
    StructField("stops", StringType, false),
    StructField("equipment", StringType, true),
))

val stream_df1 = spark.readStream
    .option("header","true")
    .schema(schema)
    .csv("file:///home/jovyan/out/routes")

// // The special value \N is used for "NULL" to indicate that no value is available
// val stream_df2 = cols.foldLeft(stream_df1)((df: DataFrame, column: String) => df.withColumn(column, when(col(column) === "\\N",lit(null)).otherwise(col(column))))
// // cast numeric columns to int
// val numericCols = Seq("airlineId", "srcAirportId", "destAirportId", "stops")
// val stream_df3 = numericCols.foldLeft(stream_df2)((df: DataFrame, column: String) => df.withColumn(column,col(column).cast(IntegerType)))
// val stream_ds = stream_df3.as[FlightRoute]

stream_df1
// stream_ds
    .writeStream
    .format("console")
    .option("checkpointLocation", "/tmp/checkpoints/")
    .start()
    .awaitTermination()

// val top10 = ds.groupBy("srcAirport").count().sort(col("count").desc).limit(10)
// top10.write.mode("overwrite").option("header",true)
//    .csv("file:///home/jovyan/out/top10_stream.csv")



org.apache.spark.sql.streaming.StreamingQueryException:  Query [id = 93856bc0-2ee5-4222-af42-68790491a5a3, runId = 8c5cc0c4-24df-4efe-a32e-e80d43f1bd29] terminated with exception: Wrong basePath file:/home/jovyan/out/routes for the root path: file:/home/jovyan/routes/part-00000-48828e63-bd5f-4ae9-bf16-10947c6b2c55-c000.csv

In [ ]:
// Next change your streaming job so the aggregations are done using sliding windows. Pick any
// window and sliding interval. The end result should be the top 10 airports used as source airport
// within each window. When choosing the window interval, keep the size of the dataset in mind.

.groupBy(
  window($"timestamp", "10 minutes", "5 minutes"),
  $"word"
)


In [ ]:
// Productionize your code by adding unit tests.

In [ ]:
// description of your steps and thinking process